In [1]:
import time
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision.models as models

MLP class

In [2]:
class MLP:
    def __init__(self, size, add_bias, activation_function):
        self.size = size
        self.depth = len(size)
        self.weights = []
        self.activation_function = activation_function
        self.add_bias = add_bias

        for i in range(self.depth - 1):
            self.weights.append(np.random.normal(scale=0.1, size=(self.size[i], self.size[i + 1])))

        if self.add_bias:
            self.weights[0] = np.row_stack([self.weights[0], np.ones(self.weights[0].shape[1])])

    def predict(self, x):
        yh = x
        for i in range(self.depth - 2):
            yh = self.activation_function[0](np.dot(yh, self.weights[i]))
        return MLP.softmax(np.dot(yh, self.weights[-1]))

    def fit(self, x_train, y_train, learning_rate=0.1, max_iters=1000,
            mini_batch_size=32, epsilon=1e-8, add_regularization="None", lambdaa=0.01):
        num_batches = len(x_train) // mini_batch_size

        norms = np.array([np.inf])
        t = 1
        while np.any(norms > epsilon) and t < max_iters:
            # Shuffle the training data
            indices = np.random.permutation(len(x_train))
            x_train = x_train[indices]
            y_train = y_train[indices]

            # Train on each batch
            for batch in range(num_batches):
                # Select a batch of data
                start = batch * mini_batch_size
                end = (batch + 1) * mini_batch_size
                x_batch = x_train[start:end]
                y_batch = y_train[start:end]
                # print(x_batch.shape, y_batch.shape)
                grad = self.gradient(x_batch, y_batch, add_regularization, lambdaa)

                # Update the weights using stochastic gradient descent
                for i in range(len(self.weights)):
                    self.weights[i] -= learning_rate * grad[i]

                t += 1
                norms = np.array([np.linalg.norm(g) for g in grad])

    def gradient(self, x, y, add_regularization, lambdaa):
        # Forward pass
        zs = [x]
        z = x
        for i in range(self.depth - 2):
            z = np.dot(z, self.weights[i])
            zs.append(z)
            z = self.activation_function[0](z)
        z = np.dot(z, self.weights[-1])
        zs.append(z)

        yh = MLP.softmax(z)

        # Backward pass
        delta = yh - y

        grads = [None] * (self.depth - 1)
        for i in range(len(grads) - 1, -1, -1):
            grads[i] = np.dot(self.activation_function[0](zs[i]).T, delta) / x.shape[0]
            if add_regularization == "L1":
                grads[i][1:] += lambdaa * np.sign(self.weights[i][1:])
            elif add_regularization == "L2":
                grads[i][1:] += lambdaa * self.weights[i][1:]
            delta = np.dot(delta, self.weights[i].T)
            delta = delta * self.activation_function[1](self.activation_function[0](zs[i]))

        return grads

    @staticmethod
    def softmax(x):
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

    @staticmethod
    def evaluate_acc(yh, y):
        correct = 0
        for i in range(len(yh)):
            if np.argmax(yh[i]) == np.argmax(y[i]):
                correct += 1

        return round(correct / len(yh), 3)


In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data


def translate(label):
    y_train = []
    for c in label:
        temp = [0 for i in range(10)]
        temp[c] = 1
        y_train.append(temp)
    return np.array(y_train)


def training_data():
    data = unpickle("data_batch_1")
    x1 = data[b'data']
    y1 = translate(data[b'labels'])

    data = unpickle("data_batch_2")
    x2 = data[b'data']
    y2 = translate(data[b'labels'])

    data = unpickle("data_batch_3")
    x3 = data[b'data']
    y3 = translate(data[b'labels'])

    data = unpickle("data_batch_4")
    x4 = data[b'data']
    y4 = translate(data[b'labels'])

    data = unpickle("data_batch_5")
    x5 = data[b'data']
    y5 = translate(data[b'labels'])

    x_train = x1
    x_train = np.vstack((x_train, x2))
    x_train = np.vstack((x_train, x3))
    x_train = np.vstack((x_train, x4))
    x_train = np.vstack((x_train, x5))

    y_train = y1
    y_train = np.vstack((y_train, y2))
    y_train = np.vstack((y_train, y3))
    y_train = np.vstack((y_train, y4))
    y_train = np.vstack((y_train, y5))

    return x_train, y_train


def normalize(X_train, X_test):
    """
    Normalize the training and test sets.

    Parameters:
    X_train (numpy array): The training set of shape `(n_train, d)`.
    X_test (numpy array): The test set of shape `(n_test, d)`.

    Returns:
    X_train_norm (numpy array): The normalized training set.
    X_test_norm (numpy array): The normalized test set.
    """
    # Compute mean and standard deviation of the training set
    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)
    # print(mean.shape, std.shape)
    # Normalize training set and test set
    X_train_norm = (X_train - mean) / std
    X_test_norm = (X_test - mean) / std

    return X_train_norm, X_test_norm

Activation functions

In [4]:
def relu(x):
    return np.maximum(0, x)


def relu_derivative(x):
    return (x > 0).astype(int)


def tanh(x):
    return np.tanh(x)


def tanh_derivative(x):
    return 1 - np.tanh(x)**2


def leaky_relu(x, alpha=0.1):
    return np.maximum(alpha * x, x)


def leaky_relu_derivative(x, alpha=0.1):
    dx = np.ones_like(x)
    dx[x < 0] = alpha
    return dx


def linear(x):
    return x


def linear_derivative(x):
    return np.ones_like(x)

Plot functions

In [9]:
def grid_search():
    lrs = [0.0005, 0.001, 0.003, 0.005, 0.01, 0.03, 0.05]
    max_iters = [10000, 50000, 100000]

    plt.figure()
    plt.xlabel("Max iterations")
    plt.ylabel("Accuracy")

    plt.title("Accuracy in different learning rate")

    for lr in lrs:
        acc_lr = []
        for max_iter in max_iters:
            mlp = MLP([3072, 10], ADD_BIAS, [linear, linear_derivative])

            start_time = time.time()
            mlp.fit(X_TRAIN, Y_TRAIN, learning_rate=lr, max_iters=max_iter, mini_batch_size=32)
            end_time = time.time()

            y_predict = mlp.predict(X_TEST)
            acc = round(mlp.evaluate_acc(y_predict, Y_TEST), 3) * 100
            acc_lr.append(acc)
            print("(", lr, max_iter, ") done, acc =", acc, "%, runtime = ", round(end_time - start_time, 3), "s")
        plt.plot(max_iters, acc_lr)

    plt.legend(lrs)
    plt.show()


def regularization():
    lambdaas = [0.01, 0.05, 0.1, 0.5, 1]
    regularizations = ["None", "L1", "L2"]

    plt.figure()
    plt.xlabel("Lambda")
    plt.ylabel("Accuracy")

    plt.title("Accuracy in different regularization and lambda")

    for re in regularizations:
        accs = []
        for lambdaa in lambdaas:
            mlp = MLP([3072, 256, 256, 10], ADD_BIAS, [relu, relu_derivative])

            start_time = time.time()
            mlp.fit(X_TRAIN, Y_TRAIN,
                    learning_rate=0.005, max_iters=50000, mini_batch_size=32, add_regularization=re, lambdaa=lambdaa)
            end_time = time.time()

            y_predict = mlp.predict(X_TEST)
            acc = round(mlp.evaluate_acc(y_predict, Y_TEST), 3) * 100
            accs.append(acc)
            print("(", re, lambdaa, ") done, acc =", acc, "%, runtime = ", round(end_time - start_time, 3), "s")
        plt.plot(lambdaas, accs)

    plt.legend(regularizations)
    plt.show()


def normalized_data():
    x_train, y_train = training_data()
    x_test = TEST_DATA[b'data']

    # Non-normalized data
    mlp = MLP([3072, 256, 256, 10], False, [relu, relu_derivative])
    mlp.fit(x_train, y_train,
            learning_rate=0.005, max_iters=50000, mini_batch_size=32, add_regularization="None")
    y_predict = mlp.predict(x_test)
    acc_non_normalized = round(mlp.evaluate_acc(y_predict, Y_TEST), 3) * 100

    # Normalized data
    x_train, x_test = normalize(x_train, x_test)
    mlp = MLP([3072, 256, 256, 10], False, [relu, relu_derivative])
    mlp.fit(x_train, y_train,
            learning_rate=0.005, max_iters=50000, mini_batch_size=32, add_regularization="None")
    y_predict = mlp.predict(x_test)
    acc_normalized = round(mlp.evaluate_acc(y_predict, Y_TEST), 3) * 100

    plt.figure()
    plt.ylabel("Accuracy")

    plt.title("Normalized data")
    plt.bar(["Non-normalized", "Normalized"], [acc_non_normalized, acc_normalized])
    plt.show()


def activation_function_selection():
    activation_functions_name = ["ReLU", "leaky-ReLU", "tanh"]
    activation_functions = [[relu, relu_derivative], [leaky_relu, leaky_relu_derivative], [tanh, tanh_derivative]]

    accs = []
    mlp = MLP([3072, 256, 256, 10], ADD_BIAS, [])
    init_weights = mlp.weights[:]
    for func in activation_functions:
        mlp.activation_function = func
        mlp.weights = init_weights[:]

        start_time = time.time()
        mlp.fit(X_TRAIN, Y_TRAIN,
                learning_rate=0.005, max_iters=50000, mini_batch_size=32, add_regularization="None")
        end_time = time.time()

        y_predict = mlp.predict(X_TEST)
        acc = round(mlp.evaluate_acc(y_predict, Y_TEST), 3) * 100
        accs.append(acc)
        print(func, "done, acc =", acc, "%, runtime = ", round(end_time - start_time, 3), "s")

    plt.figure()
    plt.ylabel("Accuracy")

    plt.title("Accuracy in different activation function")
    plt.bar(activation_functions_name, accs)
    plt.show()


def different_width():
    widths = ["64", "128", "256", "512", "1024"]
    accs = []

    for width in widths:
        mlp = MLP([3072, int(width), 10], ADD_BIAS, [relu, relu_derivative])
        start_time = time.time()
        mlp.fit(X_TRAIN, Y_TRAIN,
                learning_rate=0.001, max_iters=100000, mini_batch_size=32, add_regularization="None")
        end_time = time.time()

        y_predict = mlp.predict(X_TEST)
        acc = round(mlp.evaluate_acc(y_predict, Y_TEST), 3) * 100
        accs.append(acc)
        print(width, "done, acc =", acc, "%, runtime = ", round(end_time - start_time, 3), "s")

    plt.figure()
    plt.xlabel("Width")
    plt.ylabel("Accuracy")

    plt.title("Accuracy in different widths")
    plt.plot(widths, accs)
    plt.show()


def different_depth():
    accs = []

    # no hidden layer
    mlp = MLP([3072, 10], ADD_BIAS, [linear, linear_derivative])
    start_time = time.time()
    mlp.fit(X_TRAIN, Y_TRAIN,
            learning_rate=0.005, max_iters=50000, mini_batch_size=32, add_regularization="None")
    end_time = time.time()
    y_predict = mlp.predict(X_TEST)
    acc = round(mlp.evaluate_acc(y_predict, Y_TEST), 3) * 100
    accs.append(acc)
    print("Linear done, acc =", acc, "%, runtime = ", round(end_time - start_time, 3), "s")

    # 1 hidden layer
    mlp = MLP([3072, 256, 10], ADD_BIAS, [relu, relu_derivative])
    start_time = time.time()
    mlp.fit(X_TRAIN, Y_TRAIN,
            learning_rate=0.001, max_iters=100000, mini_batch_size=32, add_regularization="None")
    end_time = time.time()
    y_predict = mlp.predict(X_TEST)
    acc = round(mlp.evaluate_acc(y_predict, Y_TEST), 3) * 100
    accs.append(acc)
    print("1 hidden done, acc =", acc, "%, runtime = ", round(end_time - start_time, 3), "s")

    # 2 hidden layer
    mlp = MLP([3072, 256, 256, 10], ADD_BIAS, [relu, relu_derivative])
    start_time = time.time()
    mlp.fit(X_TRAIN, Y_TRAIN,
            learning_rate=0.005, max_iters=50000, mini_batch_size=32, add_regularization="None")
    end_time = time.time()
    y_predict = mlp.predict(X_TEST)
    acc = round(mlp.evaluate_acc(y_predict, Y_TEST), 3) * 100
    accs.append(acc)
    print("2 hidden done, acc =", acc, "%, runtime = ", round(end_time - start_time, 3), "s")

    plt.figure()
    plt.xlabel("Number of hidden layer")
    plt.ylabel("Accuracy")

    plt.title("Accuracy in different number of hidden layers")
    plt.bar(["0", "1", "2"], accs)
    plt.show()


def run():
    mlp = MLP([3072, 256, 10], ADD_BIAS, [relu, relu_derivative])

    start_time = time.time()
    print("start time:", start_time)
    mlp.fit(X_TRAIN, Y_TRAIN,
            learning_rate=0.005, max_iters=50000, mini_batch_size=32, add_regularization="None", lambdaa=0.01)

    end_time = time.time()
    print("end time:", end_time)

    Y_PREDICT = mlp.predict(X_TEST)
    print("y_predict: ", Y_PREDICT[0])
    print("y_test: ", Y_TEST[0])
    ACC = mlp.evaluate_acc(Y_PREDICT, Y_TEST)
    print("acc = ", ACC * 100, "%\nruntime: ", round(end_time - start_time, 3), "s")


def evaluate_acc(yh, y):
    correct = 0
    for i in range(len(yh)):
        if np.argmax(yh[i]) == np.argmax(y[i]):
            correct += 1
    return round(correct / len(yh), 3)


def train(eps, lr, dl, net):
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    for t in range(eps):
        for i, data in enumerate(dl, 0):
            inputs, labels = data
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()

            optimizer.zero_grad()

            outputs = net(inputs)
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()


def train_and_plt_cnn(max_es, l_rates, d_loader):
    plt.figure()
    plt.xlabel("Max Epochs")
    plt.ylabel("Accuracy")
    plt.title("Accuracy in different learning rate in CNN")

    for lr in l_rates:
        acc_lr = []
        for e in max_es:
            start = time.time()
            model = CNN(outputs=10)
            model.to(device)
            train(int(e), lr, d_loader, model)
            y_pred = model(x_test_tensor)
            y_pred = y_pred.cpu()
            y_pred = y_pred.detach().numpy()
            accuracy = round(evaluate_acc(y_pred, Y_TEST), 3) * 100
            acc_lr.append(accuracy)
            end = time.time()
            running_time = end - start
            print("hyperparams: " + str(e) + ' ' + str(lr))
            print("Time elapsed: " + str(running_time))
        plt.plot(epochs, acc_lr)

    plt.legend(lrs)
    plt.show()


def train_and_plt_res(max_es, l_rates, d_loader):
    plt.figure()
    plt.xlabel("Max Epochs")
    plt.ylabel("Accuracy")
    plt.title("Accuracy in different learning rate in pretrained RESnet")

    for lr in l_rates:
        acc_lr = []
        for e in max_es:
            start = time.time()

            resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

            for param in resnet.parameters():
                param.requires_grad = False

            num_features = resnet.fc.in_features
            resnet.fc = nn.Sequential(
                nn.Linear(num_features, 256),
                nn.ReLU(inplace=True),
                nn.Linear(256, 256),
                nn.ReLU(inplace=True),
                nn.Linear(256, 10),
                nn.Softmax(dim=1)
            )

            resnet.to(device)
            train(int(e), lr, d_loader, resnet)
            y_pred = resnet(x_test_tensor)
            y_pred = y_pred.cpu()
            y_pred = y_pred.detach().numpy()
            accuracy = round(evaluate_acc(y_pred, Y_TEST), 3) * 100
            acc_lr.append(accuracy)
            end = time.time()
            running_time = end - start
            print("hyperparams: " + str(e) + ' ' + str(lr))
            print("Time elapsed: " + str(running_time))
        plt.plot(epochs, acc_lr)

    plt.legend(lrs)
    plt.show()


def train_and_plot_time_diff(d_loader):
    plt.figure()
    plt.ylabel("Training Time")
    plt.title("Training Time Difference Between CNN and Pretrained ResNet Model")

    model_names = ['CNN', 'Pretrained ResNet']
    training_times = []

    cnn = CNN(outputs=10)
    cnn.to(device)

    start = time.time()

    train(50, 0.05, d_loader, cnn)

    end = time.time()

    cnn_training_time = end - start
    training_times.append(cnn_training_time)

    resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    for param in resnet.parameters():
        param.requires_grad = False
    num_features = resnet.fc.in_features
    resnet.fc = nn.Sequential(
        nn.Linear(num_features, 256),
        nn.ReLU(inplace=True),
        nn.Linear(256, 256),
        nn.ReLU(inplace=True),
        nn.Linear(256, 10),
        nn.Softmax(dim=1)
    )
    resnet.to(device)

    start = time.time()

    train(50, 0.05, d_loader, resnet)

    end = time.time()

    resnet_training_time = end - start
    training_times.append(resnet_training_time)

    plt.bar(model_names, training_times)
    plt.show()

Pytorch CNN

In [6]:
class CNN(nn.Module):
    def __init__(self, outputs=10, hidden1=256, hidden2=256):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        # Second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.fc1 = nn.Linear(64*8*8, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, outputs)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x


Main

In [ ]:
if __name__ == '__main__':
    TEST_DATA = unpickle("test_batch")

    ADD_BIAS = False
    USING_CNN = False
    USING_RESNET = False
    X_TRAIN, Y_TRAIN = training_data()
    X_TEST = TEST_DATA[b'data']
    X_TRAIN, X_TEST = normalize(X_TRAIN, X_TEST)

    Y_TEST = translate(TEST_DATA[b'labels'])

    if ADD_BIAS:
        X_TRAIN = np.column_stack([X_TRAIN, np.ones(X_TRAIN.shape[0])])


    different_depth()
    if USING_CNN:
        X_TRAIN = X_TRAIN.reshape(-1, 3, 32, 32)

        X_TEST = X_TEST.reshape(-1, 3, 32, 32)

        x_train_tensor = Variable(torch.from_numpy(X_TRAIN).float())
        y_train_tensor = Variable(torch.from_numpy(Y_TRAIN).float())
        x_test_tensor = Variable(torch.from_numpy(X_TEST).float())

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        if torch.cuda.is_available():
            x_test_tensor = x_test_tensor.cuda()

        dataset = torch.utils.data.TensorDataset(torch.tensor(x_train_tensor), torch.tensor(y_train_tensor))

        loader = DataLoader(dataset, batch_size=32, shuffle=True)

        lrs = [0.03, 0.05]
        epochs = ['5', '10']

        train_and_plt_cnn(epochs, lrs, loader)
    
    elif USING_RESNET:
      X_TRAIN = X_TRAIN.reshape(-1, 3, 32, 32)

      X_TEST = X_TEST.reshape(-1, 3, 32, 32)

      x_train_tensor = Variable(torch.from_numpy(X_TRAIN).float())
      y_train_tensor = Variable(torch.from_numpy(Y_TRAIN).float())
      x_test_tensor = Variable(torch.from_numpy(X_TEST).float())

      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

      if torch.cuda.is_available():
        x_test_tensor = x_test_tensor.cuda()

      dataset = torch.utils.data.TensorDataset(torch.tensor(x_train_tensor), torch.tensor(y_train_tensor))

      loader = DataLoader(dataset, batch_size=32, shuffle=True)

      lrs = [0.01, 0.03, 0.05, 0.07, 0.1]
      epochs = ['5', '10', '20', '50']

      train_and_plt_res(epochs, lrs, loader)
      # train_and_plot_time_diff(loader)





Linear done, acc = 36.199999999999996 %, runtime =  97.98 s
1 hidden done, acc = 45.300000000000004 %, runtime =  1763.957 s
